<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/RAG_app/RAG_app_law_QA_ReAct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install langchain_community gradio openai chromadb tiktoken langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from google.colab import files
import os

uploaded = files.upload()
filename = list(uploaded.keys())[0]

# JSON 파일 읽기
with open(filename, 'r') as json_file:
    data = json.load(json_file)

os.environ["OPENAI_API_KEY"] = data.get("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = data.get("TAVILY_API_KEY")

Saving api_keys.json to api_keys (2).json


In [6]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters.base import Language
from langchain_community.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.chat_models import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import (
    HumanMessage,AIMessage
)

import gradio as gr
import warnings
import json
import openai

warnings.filterwarnings('ignore')

In [7]:
class ChromaEmbedding:
    def __init__(self, directory, embedding):
        """
        :param directory: 벡터 데이터베이스 폴더
        :param embedding: 임베딩을 수행할 모델
        """
        self.directory = directory
        self.chromaDb = Chroma(persist_directory=self.directory, embedding_function=embedding)

    def addJSONL(self, jsonl_file):

        with open(jsonl_file, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        documents = []
        for line in lines:
            data = json.loads(line)
            question = data.get("question", "")
            answer = data.get("answer", "")
            content = f"Question: {question}\nAnswer: {answer}"
            documents.append(content)

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=900,
            chunk_overlap=0,
            length_function=len,
        )

        # 문자열을 지정된 크기의 청크로 분할한다.
        docs = text_splitter.create_documents(documents)

        # 분할된 청크를 임베딩 데이터베이스에 저장한다.
        self.chromaDb.add_documents(docs)
        self.chromaDb.persist()

In [8]:
# 벡터 데이터베이스 폴더
INDEX = "/content/drive/MyDrive/AI-modeling/law_RAG app/app_QA_index"

def buildIndex():
    chroma = ChromaEmbedding(INDEX, OpenAIEmbeddings(openai_api_key = openai_api_key))
    chroma.addJSONL("/content/drive/MyDrive/AI-modeling/law_RAG app/data/law_qa_sample.jsonl")
    print("임베딩 완료!")

In [15]:
def runApplication():
    """
    RAG 어플리케이션을 실행한다.
    :return:
    """
    chroma = ChromaEmbedding(INDEX, OpenAIEmbeddings(openai_api_key = openai_api_key))
    retriever = chroma.chromaDb.as_retriever(
        search_type="similarity",
        search_kwargs={
            'k': 2,  # 리턴 문서 수
        }
    )

    # llm
    llm=ChatOpenAI(
        streaming=True,
        callbacks=[StreamingStdOutCallbackHandler()],
        model="gpt-3.5-turbo-0125",
        temperature=0,
        openai_api_key = openai_api_key,
    )

    # QA
    qa_interface = RetrievalQA.from_chain_type(llm=llm,
                                               chain_type="stuff",
                                               retriever=retriever,
                                               chain_type_kwargs={
                                                   "verbose": True,
                                               },
                                               return_source_documents=True)


    with gr.Blocks() as rag_tester:
        gr.HTML("<h2>법률 사례기반 챗봇</h2>")

        with gr.Row():
            with gr.Column(scale=1):
                chatbot = gr.Chatbot()
                msg = gr.Textbox(value="법률 관련 질문을 입력하세요.")
            with gr.Column(scale=1):
                gr.HTML("<h3>검색 문서")
                source = gr.Textbox()

        def userHandler(user_message, history):
            return "", history + [[user_message, None]]

        def botHandler(history):
            qa_result = qa_interface(history[-1][0])

            history[-1][1] = qa_result["result"]
            source_text = ""
            for ix, doc in enumerate(qa_result['source_documents']):
                source_text += f"## 검색문서 {ix+1}\n```\n{doc.page_content}\n```\n\n"

            for i in range(len(history)):
                print(f"### {i}번째\nQuestion: {history[i][0]}\nAnswer: {history[i][1]}")

            return history, source_text

        msg.submit(userHandler, [msg, chatbot], [msg, chatbot], queue=False).then(
          botHandler, chatbot, [chatbot, source]
        )

    rag_tester.launch(debug=True, share=True)

if __name__ == '__main__':
    # 임베딩 폴더가 없는 경우 임베딩을 수행한다.
    # if not os.path.isdir(INDEX):
        # buildIndex()
        runApplication()

NotImplementedError: 

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.prompts import MessagesPlaceholder

# llm
llm=ChatOpenAI(
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model="gpt-3.5-turbo-0125",
    temperature=0
)

# prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            당신은 법률기반 질의응답에 기반해 사용자의 질문에 답변하는 챗봇입니다.
            반드시 전달받은 context에 기반해 답변해야 하며,
            전달받은 context로 답변할 수 없을 시 답변을 절대 하면 안 됩니다.
            Lets' think step by step.
            """
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad") # 메세지 리스트를 전달하는 공간
    ]
)

# retriever
chroma = ChromaEmbedding(INDEX, OpenAIEmbeddings())
retriever = chroma.chromaDb.as_retriever(
    search_type="similarity",
    search_kwargs={
        'k': 2,  # 리턴 문서 수
    }
)
retriever_tool = create_retriever_tool(
    retriever,
    "law_qa_search",
    "법률에 대한 질문이라면 반드시 이 tool을 사용하세요.",
)

# search
search = TavilySearchResults(max_results=1)

# tools
# tools = [retriever_tool]
tools = [retriever_tool, search]

# # 언어체인
# combine_docs_chain = create_stuff_documents_chain(llm, prompt)
# # 검색체인
# retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result1 = agent_executor.invoke({"input": "근로기준법상 근로자에는 정규직만 들어가는 건가요?"})
result2 = agent_executor.invoke({"input": "연예인 뉴진스에 대해 설명해주세요."})



> Entering new AgentExecutor chain...

Invoking: `law_qa_search` with `{'query': '근로기준법 근로자 정의'}`


Answer: 「근로기준법」제2조 제1항 제1호는 “‘근로자’란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 자를 말한다.”라고 규정하고 있습니다.근로자의 범위에 관하여 판례는 “근로기준법상의 근로자에 해당하는지 여부를 판단함에 있어서는 그 계약의 형식이 민법상의 고용계약인지 또는 도급계약인지에 관계없이 그 실질에 있어 근로자가 사업 또는 사업장에 임금을 목적으로 종속적인 관계에서 사용자에게 근로를 제공하였는지 여부에 따라 판단하여야 할 것이고, 위에서 말하는 종속적인 관계가 있는지 여부를 판단함에 있어서는, 업무의 내용이 사용자에 의하여 정하여지고 취업규칙 또는 복무(인사)규정 등의 적용을 받으며 업무수행과정에 있어서도 사용자로부터 구체적, 개별적인 지휘·감독을 받는지 여부, 사용자에 의하여 근무시간과 근무장소가 지정되고 이에 구속을 받는지 여부, 근로자 스스로가 제3자를 고용하여 업무를 대행케 하는 등 업무의 대체성 유무, 비품, 원자재나 작업도구 등의 소유관계, 보수의 성격이 근로 자체의 대상적 성격이 있는지 여부와 기본급이나 고정급이 정하여져 있는지 여부 및 근로소득세의 원천징수 여부 등 보수에 관한 사항, 근로제공관계의 계속성과 사용자에의 전속성의 유무와 정도, 사회보장제도에 관한 법령 등 다른 법령에 의하여 근로자로서의 지위를 인정받는지 여부, 양 당사자의 경제·사회적 조건 등을 종합적으로 고려하여 판단하여야 할 것이다.”라고 하였습니다(대법원 1994. 12. 9. 선고 94다22859 판결).한편, 위 사안과 관련하여 판례는 “학교법인이 운영하는 대학교에서 강의를 담당한 시간강사들은 학교측에서 시간강사들의 위촉·재위촉과 해촉 또는 해임, 강의시간 및 강사료, 시간강사의 권리와 의무 등에 관하여 정한 규정에 따라 총장 등에 의하여 시간강사로

In [62]:
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

functions = [format_tool_to_openai_function(f) for f in tools]
model = llm.bind(functions=functions)

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

result1 = chain.invoke({
    "input": "근로기준법상 근로자에는 정규직만 들어가는 건가요?",
    "agent_scratchpad": []
})

print(result1.tool) # law_qa_search
print(result1.tool_input) # {'query': '근로기준법 근로자 정의'
result1.message_log # [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"근로기준법 근로자 정의"}', 'name': 'law_qa_search'}}, response_metadata={'finish_reason': 'function_call'}, id='run-0e628874-3958-436d-924b-1e33b55c5ce8-0')]

law_qa_search
{'query': '곰'}


[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"곰"}', 'name': 'law_qa_search'}}, response_metadata={'finish_reason': 'function_call'}, id='run-497a8198-6d9f-4075-83a8-9ab059f4d46c-0')]

In [ ]:

# 결과
[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}'}})]

# 이 메시지와 observation을 scratchpad에 전달해준다.
format_to_openai_functions([(result1, obseravation), ]) # 메시지와 observation의 튜플을 넘겨주고, 반복하기 때문에 컴마를 붙여줌
# 결과
[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}'}}),
 FunctionMessage(content='The current temperature is 11.3°C', name='get_current_temperature')]